In [1]:
import pandas as pd
import numpy as np

In [2]:
# read drug basic data
df1 = pd.read_excel('1.藥品基本檔.xlsx')
df2 = pd.read_excel('2.藥物諮詢.xlsx')
exclude = pd.read_excel('exclude.xlsx')

In [3]:
# drug selection
df1 = df1[(df1['藥品狀態'] == '可用')]

exclude_list = ['廠商缺貨，可查類似藥', '停用', '廠商缺貨', '停用，可查類似藥']
df1 = df1[~df1['DC註記'].isin(exclude_list)]

exclude_list = exclude.iloc[:,0].tolist()
df1 = df1[~df1['藥品代碼'].isin(exclude_list)]  # exclude drugs in exclusion list

df1 = df1.dropna(subset=['藥理分類2'])  # exclude empty & weird category 2
exclude_list2 = ['MEDD', 'ZOTH', 'PHR']
df1 = df1[~df1['藥理分類2'].isin(exclude_list2)]

In [4]:
df1 = df1[['藥品代碼', '商品英文名稱', '商品學名', '藥理分類1', '藥理分類2', 'DC註記']]
df2 = df2[['藥品代碼', '適應症', '用法用量', '肝功能異常(Y/N)', '腎功能異常(Y/N)', '禁忌', '副作用', '孕期用藥建議', '哺乳期用藥建議']]
df = df1.merge(df2, on='藥品代碼', how='left')

In [5]:
df = df.rename(columns={'肝功能異常(Y/N)': '肝功能異常', '腎功能異常(Y/N)': '腎功能異常', '孕期用藥建議': '孕期建議', '哺乳期用藥建議': '哺乳期建議'})
df = df.replace(np.nan, 'No Data')
df['DC註記'] = df['DC註記'].replace('No Data', '')
df['DC註記'] = df['DC註記'].replace('臨採藥，請通知藥局外購', '臨採')

In [6]:
# make file directories
import os

try:
    path = 'C:\\Users\\152551\\formulary-gitbook\\toc'
    # path = '/Users/shin/formulary/toc'
    os.mkdir(path)
except:
    pass
cat2_li = df['藥理分類2'].unique().tolist()
for cat in cat2_li:
    if cat[-3] == '-':
        try:
            path = 'C:\\Users\\152551\\formulary-gitbook\\toc\\' + \
            cat[:3].lower() + '00-00\\'
            # path = '/Users/shin/formulary/toc/' + cat[:3].lower() + '00-00/'
            os.mkdir(path)
        except:
            pass
    else:
        pass

for cat in cat2_li:
    if cat[-2:] == "00":
        try:
            path = 'C:\\Users\\152551\\formulary-gitbook\\toc\\' + \
            cat[:3].lower() + '00-00\\' + cat.lower() + '\\'
            # path = '/Users/shin/formulary/toc/' + cat[:3].lower() + '00-00/' + cat.lower() + '/'
            os.mkdir(path)
        except:
            pass
    elif (cat[-2:] != "00") & (len(cat) == 8):
        try:
            path = 'C:\\Users\\152551\\formulary-gitbook\\toc\\' + \
            cat[:3].lower() + '00-00\\' + cat[:6].lower() + '00\\'
            os.mkdir(path)
        except:
            pass
        try:
            path = 'C:\\Users\\152551\\formulary-gitbook\\toc\\' + \
            cat[:3].lower() + '00-00\\' + cat[:6].lower() + '00\\' + cat.lower() + '\\'
            # path = '/Users/shin/formulary/toc/' + cat[:3].lower() + '00-00/' + cat[:6].lower() + '00/'+ cat.lower() + '/'
            os.mkdir(path)
        except:
            pass
    else:
        try:
            path = 'C:\\Users\\152551\\formulary-gitbook\\toc\\' + cat.lower() + '\\'
            # path = '/Users/shin/formulary/toc/' + cat.lower()
            os.mkdir(path)
        except:
            pass

In [7]:
# add path to each drug (to save markdown files)
toc1 = []
toc2 = []
toc3 = []
for i in df['藥理分類2'].tolist():
    if i[-2:] == '00':
        toc1.append(i[:2].lower() + r'-00-00')
        toc2.append(i.lower())
        toc3.append(np.nan)
    elif (i[-2:] != "00") & (len(i) == 8):
        toc1.append(i[:2].lower() + r'-00-00')
        toc2.append(i[:5].lower() + r'-00')
        toc3.append(i.lower())
    else:
        toc1.append(i.lower())
        toc2.append(np.nan)
        toc3.append(np.nan)
df['toc1'] = toc1
df['toc2'] = toc2
df['toc3'] = toc3

df['drug_name'] = df['商品學名'].str.lower()
li = df['drug_name'].tolist()
new_li = []
for i in li:
    i = i.replace(' + ', '_and_')
    i = i.replace(' & ', '_and_')
    i = i.replace('+', '_and_')
    i = i.replace('/', '-')
    i = i.replace(' ', '_')
    new_li.append(i)
df['drug_name'] = new_li
df['name_md'] = df['drug_name'] + '.md'

In [8]:
def func2(row):
    if pd.isna(row.toc2) & pd.isna(row.toc3):
        string = ['C:\\Users\\152551\\formulary-gitbook\\toc',
                  row['toc1'], row['name_md']]
#         string = ['/Users/shin/formulary/toc', row['toc1'], row['name_md']]
        return "\\".join(string)
    elif pd.notna(row.toc3):
        string = ['C:\\Users\\152551\\formulary-gitbook\\toc',
                  row['toc1'], row['toc2'], row['toc3'], row['name_md']]
#         string = ['/Users/shin/formulary/toc', row['toc1'], row['toc2'], row['toc3'], row['name_md']]
        return "\\".join(string)
    else:
        string = ['C:\\Users\\152551\\formulary-gitbook\\toc',
                  row['toc1'], row['toc2'], row['name_md']]
#         string = ['/Users/shin/formulary/toc', row['toc1'], row['toc2'], row['name_md']]
        return "\\".join(string)


df['url'] = df.apply(func2, axis=1)
df = df.sort_values(by=['藥理分類2'])

In [10]:
# write markdown files and save
cat2_li = df['藥理分類2'].unique().tolist()

# go through Category 2
for cat2 in cat2_li:
    df_cat2 = df[df['藥理分類2'] == cat2]
    
    # go through drug names under each Category 2
    names = df_cat2['商品學名'].unique().tolist()
    for name in names:
        df_name = df_cat2[df_cat2['商品學名'] == name]
        df_name = df_name.sort_values(by=['藥品代碼'], ascending=True).reset_index(drop=True)
        # find drug codes with a same drug name, and go through codes with a same drug name
        code_list = df_name['藥品代碼'].tolist()
        df_name.set_index('藥品代碼', inplace=True)
        # because drugs with a same drug name would be written in a same file, the saving path is the same
        filepath = df_name.iloc[0]['url']
        with open(filepath, "w", encoding='utf-8') as f:
            title1 = '# ' + name + '\n\n'
            f.write(title1)
            for code in code_list:
                prodname = df_name.loc[code, '商品英文名稱']
                title2 = '## ' + prodname + '\n\n'
                f.write(title2)

                note = df_name.loc[code, 'DC註記']
                try:
                    title3 = '##### ' + note + '\n\n'
                except:
                    title3 = ''
                f.write(title3)

                df_tb = df_name.loc[code,'適應症':'哺乳期建議'].reset_index()
                df_tb.columns = ['藥品代碼', code]
                content = df_tb.to_markdown(index=0)
                f.write(content)

                f.write('\n\n')